In [4]:
import cv2
import os, json
import numpy as np

## Capture a background image

In [7]:
is_light = False
settings_filename = os.path.join('profiles', 'prodesk-home.json')

with open(settings_filename) as json_file:
    cfg = json.load(json_file)

In [8]:
# Define BGR colors
BGR_COLOR = {
    'red': (0,0,255),
    'green': (127,255,0),
    'blue': (255,127,0),
    'yellow': (0,127,255),
    'black': (0,0,0),
    'white': (255,255,255)
}

cfg_exp = cfg['experiment']
cfg_cam = cfg['camera']
cfg_pos = cfg['position']

cap = cv2.VideoCapture(cfg_cam['source'], cfg_cam['api']) if cfg_cam['api'] else cv2.VideoCapture(cfg_cam['source'])
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))
cap.set(cv2.CAP_PROP_FRAME_WIDTH, cfg_cam['frame_width'])
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cfg_cam['frame_height'])
cap.set(cv2.CAP_PROP_FPS, cfg_cam['fps'])
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))    

while(True):
    ret, frame = cap.read()

    # Draw the arena area
    cv2.circle(frame, (cfg_pos['arena_x'], cfg_pos['arena_y']), cfg_pos['arena_radius'], BGR_COLOR['red'], 2)

    # Mask the space outside the arena
    mask = np.zeros(shape=frame.shape, dtype="uint8")
    cv2.circle(mask, (cfg_pos['arena_x'], cfg_pos['arena_y']), cfg_pos['arena_radius'], BGR_COLOR['white'], -1)
    
    frame = cv2.bitwise_and(src1=frame, src2=mask)
    cv2.putText(frame, 'LIGHT' if is_light else 'DARK', (10, 20), cv2.FONT_HERSHEY_DUPLEX, .5, (255, 255, 255))
    cv2.imshow('Press "c" to capture, "q" to quit', frame)

    k = cv2.waitKey(33)
    if k == ord('c'):
        f_name = cfg_pos['background_light'] if is_light else cfg_pos['background_dark']
        cv2.imwrite(os.path.join('assets', f_name), frame)
        break
    if k == ord('q'):
        break        

# When the background image is captured, release the capture
cap.release()
cv2.destroyAllWindows()